In [1]:
import pandas as pd
import pyodbc

In [30]:
def extract_data_from_excel(file_path, sheet_name=0):
    """
    Extract data from an Excel file.
    Args:
        file_path (str): Path to the Excel file.
        sheet_name (str/int): Sheet name or index to extract from (default: first sheet).
    Returns:
        DataFrame: The extracted data as a pandas DataFrame.
    """
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
        print(f"Successfully extracted {len(df)} rows from Excel.")
        return df
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return None

# Step 2: Transform - Data Cleaning and Preparation 
def transform_data(df):
    """
    Apply transformations to the DataFrame if needed.
    Args:
        df (DataFrame): DataFrame to transform.
    Returns:
        DataFrame: Transformed DataFrame.
    """
    df_cleaned = df.dropna()  
    return df_cleaned

def load_data_to_sqlserver(df, server, database, table_name):
    """
    Load DataFrame into SQL Server.
    Args:
        df (DataFrame): DataFrame to load into the database.
        server (str): SQL Server name.
        database (str): Database name.
        table_name (str): Target table name in the database.
    """
    try:
        conn_str = (
    r'driver={SQL Server};'
    r'server=(local);'
    r'database=Traffic;'
    r'trusted_connection=yes;'
    )
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()

        for index, row in df.iterrows():
            columns = ', '.join(row.index)
            values = ', '.join(['?'] * len(row))
            sql = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"
            cursor.execute(sql, tuple(row))
        
        conn.commit()  
        print(f"Successfully loaded {len(df)} rows into the {table_name} table.")
    except Exception as e:
        print(f"Error loading data to SQL Server: {e}")
    finally:
        cursor.close()
        conn.close()

def etl_process(file_path, sheet_name, server, database, table_name):
    """
    Full ETL pipeline: Extract data from Excel, transform it, and load into SQL Server.
    """
    df = extract_data_from_excel(file_path, sheet_name)
    
    if df is not None:
        df_transformed = transform_data(df)
        
        load_data_to_sqlserver(df_transformed, server, database, table_name)
    else:
        print("ETL process failed during extraction.")


file_path = r'C:path\sheet.xlsx'  
sheet_name = 0  
server = r'ServerName'  
database = 'DatabaseName'
table_name = 'TabelName'

etl_process(file_path, sheet_name, server, database, table_name)


Successfully extracted 22411 rows from Excel.
Successfully loaded 21879 rows into the Incidents table.
